In [ ]:
import copy
from datetime import datetime
from itertools import product
from importlib import reload
from IPython import extract_module_locals
import pathlib

import numpy as np
import matplotlib.pyplot as plt

from exactdiag.logging_utils import setup_logging
from exactdiag.tJ_spin_half_ladder import api
from exactdiag.tJ_spin_half_ladder import configs


main_folder = pathlib.Path(extract_module_locals()[1]["__vsc_ipynb_file__"]).parents[3]
%cd {main_folder}

from examples.tJ_spin_half_ladder.energy_contributions_in_excited_state import functions as fs  # noqa: E402 - I could not make it work as a relative import.
reload(fs)

setup_logging()  # The logs seem to show up somewhat inconsistently.

In [ ]:
config = configs.Config(
    hamiltonian={
        "num_rungs": 10,
        "num_holes": 4,
        "weights": {"tl": 1, "tr": 1, "jl": 1.5, "jr": 1.5},
        "num_threads": 12,
        "symmetry_qs": {"leg": 0, "rung": 0},
    },
    eigenpair={"num_eigenpairs": 5},
    spectrum={
        "name": "spectral_function",
        "operator_symmetry_qs": None,
        "omega_max": 4,
        "omega_min": -4,
        "omega_steps": 1000,
        "num_lanczos_vecs": 250,
        "broadening": 0.05,
    },
)  # NOTE: We may mutate the config as we go.
# In the rest of the notebook, we assume the Hamiltonian
# and all spectral functions of this system
# are already saved at the standard location. Lets make sure of that now.
_ = api.get_spectral_function_spectra(config=config, limited_qs=True)

In [ ]:
j_to_ts = np.linspace(0.1, 1.5, 2)  # np.linspace(0.1, 1.5, 15)
j_to_ts = np.round(j_to_ts, 1)
num_excited_states = 4
excited_state_properties = fs.calculate_excited_state_properties(
    config=config,
    j_to_ts=j_to_ts,
    num_excited_states=num_excited_states,
    verbose=True,
)

In [ ]:
def get_figure(j_to_ts, values, **kwargs):
    return fs.plot_observable(
        j_to_ts,
        values,
        mkx=config.hamiltonian.num_rungs // 2 + 1,
        mky=2,
        num_excited=num_excited_states,
        **kwargs,
    )


# Lets select any q that can form a combination with
# energy of < gap_proximity_factor * gap
gap_proximity_factor = 1.2
important_qs, pairs = fs.select_q_combinations_forming_gap(
    j_to_ts=excited_state_properties.j_to_ts,
    energies=excited_state_properties.energies,
    gap_proximity_factor=gap_proximity_factor,
)
# important_qs[:] = 1 # uncomment to disable filtering for gap-forming states
start = 0
important_string = f"_selected{gap_proximity_factor}" if not np.all(important_qs[start:]) else ""

# In the following cells, we plot the excited state properties to see if we can spot a pattern.

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.energies[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    is_energy=True,
    ylabel="Energy [$J$]",
    converge=["top", "bot", "mid"],
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/energy{important_string}.pdf')
# plt.ylim(-1.2,1.1)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/energy_closer{important_string}.pdf')
plt.show()

In [ ]:
renorm = 1 / np.sqrt(config.hamiltonian.num_rungs * 2)
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    renorm * excited_state_properties.projected_norms[start:, :],
    ylabel="excited state norm",
    converge="top",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/norm{important_string}.pdf')
plt.show()

In [ ]:
in_qs = important_qs[start:]
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.tl_conts[start:, :],
    ylabel=r"$\Delta\left<t_\parallel\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/tl_dif{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.tr_conts[start:, :],
    ylabel=r"$\Delta\left<t_\perp\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/tr_dif{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.jl_conts[start:, :],
    ylabel=r"$\Delta\left<J_\parallel\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/jl_dif_in_t{important_string}.pdf')
plt.show()

fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.jl_conts[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    is_energy=True,
    ylabel="Energy [$J$]",
    converge=["top", "bot", "mid"],
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/jl_dif_in_j{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.jr_conts[start:, :],
    ylabel=r"$\Delta\left<J_\perp\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/jr_dif_in_t{important_string}.pdf')
plt.show()

fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.jr_conts[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    ylabel=r"$\Delta\left<J_\perp\right>$ [$J$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/jr_dif_in_j{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.t_conts[start:, :],
    ylabel=r"$\Delta\left<t_\parallel+t_\perp\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/t_dif{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.j_conts[start:, :],
    ylabel=r"$\Delta\left<J_\parallel+J_\perp\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/j_dif_in_t{important_string}.pdf')
plt.show()

fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.j_conts[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    ylabel=r"$\Delta\left<J_\parallel+J_\perp\right>$ [$J$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/j_dif_in_j{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.l_conts[start:, :],
    ylabel=r"$\Delta\left<J_\parallel+J_\perp\right>$ [$J$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/l_dif_in_t{important_string}.pdf')
plt.show()


fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.jl_conts[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    ylabel=r"$\Delta\left<t_\parallel+J_\parallel\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/l_dif_in_j{important_string}.pdf')
plt.show()


fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.l_conts[start:, :]
    / (1 + excited_state_properties.j_to_ts[start:, None, None, None, None]),
    ylabel=r"$\Delta\left<t_\parallel+J_\parallel\right>$ [$t+J$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/l_dif_in_tj{important_string}.pdf')
plt.show()

In [ ]:
fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.r_conts[start:, :],
    ylabel=r"$\Delta\left<t_\perp+J_\perp\right>$ [$t$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/r_dif_in_t{important_string}.pdf')
plt.show()

fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.r_conts[start:, :] / excited_state_properties.j_to_ts[start:, None, None, None, None],
    ylabel=r"$\Delta\left<t_\perp+J_\perp\right>$ [$j$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/r_dif_in_j{important_string}.pdf')
plt.show()

fig = get_figure(
    excited_state_properties.j_to_ts[start:],
    excited_state_properties.r_conts[start:, :]
    / (1 + excited_state_properties.j_to_ts[start:, None, None, None, None]),
    ylabel=r"$\Delta\left<t_\perp+J_\perp\right>$ [$t+J$]",
    converge="bot",
    important_qs=important_qs[start:],
)
# plt.savefig(notebook_dir+f'pic-exc_state_analysis/r_dif_in_tj{important_string}.pdf')
plt.show()

In [ ]:
# In the following cells, we combine (add or multiply) the properties
# of one excited state with k=[kx, ky] created by A_k
# with properties of second excited state with k'=[kx, k'y] created by A^\dagger_k'.


def pair_plot(**kwargs):
    return fs.plot_pair_observable(
        x=excited_state_properties.j_to_ts,
        pairs=pairs,
        mkx=config.hamiltonian.num_rungs // 2 + 1,
        mky=2,
        num_excited=num_excited_states,
        **kwargs,
    )

In [ ]:
fig = pair_plot(
    y=excited_state_properties.energies,
    ylim=None,
    is_multiplied=False,
    ylabel="pair energy gap [t]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/gap_in_t.pdf')
plt.show()


fig = pair_plot(
    y=excited_state_properties.energies / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel="pair energy gap [t]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/gap_in_j.pdf')
plt.show()

In [ ]:
renorm = 1 / np.sqrt(config.hamiltonian.num_rungs * 2)


fig = pair_plot(
    y=renorm * excited_state_properties.projected_norms,
    ylim=None,
    is_multiplied=True,
    ylabel="pair norm",
    converge=False,
)

# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/norm.pdf')
plt.show()


max_norm = np.zeros(len(j_to_ts))
for ix in range(len(excited_state_properties.j_to_ts)):
    for qx in range(config.hamiltonian.num_rungs // 2 + 1):
        for iq, (qy, qpy) in enumerate(product(range(2), range(2))):
            for ie, (exc1, exc2) in enumerate(product(range(num_excited_states), range(num_excited_states))):
                if pairs[ix, qx, qy, qpy, exc1, exc2]:
                    max_norm[ix] = max(
                        max_norm[ix],
                        excited_state_properties.projected_norms[ix, 0, qx, qy, exc1]
                        * excited_state_properties.projected_norms[ix, 1, qx, qpy, exc2],
                    )

fig = pair_plot(
    y=excited_state_properties.projected_norms / np.sqrt(max_norm[:, None, None, None, None]),
    ylim=None,
    is_multiplied=True,
    ylabel="relative pair norm",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/norm_relative.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.tl_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\parallel\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/tl_in_t.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.tr_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\perp\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/tr_in_t.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.jl_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\parallel\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/jl_in_t.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.jl_conts / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\parallel\right>$ [$J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/jl_in_j.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.jr_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\perp\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/jr_in_t.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.jr_conts / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\perp\right>$ [$J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/jr_in_j.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.t_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\parallel+ t_\perp\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/t_in_t.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.j_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\parallel+J_\perp\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/j_in_t.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.j_conts / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<J_\parallel+J_\perp\right>$ [$J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/j_in_j.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.l_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\parallel+J_\parallel\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/l_in_t.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.l_conts / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\parallel+J_\parallel\right>$ [$J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/l_in_j.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.l_conts / (1 + excited_state_properties.j_to_ts[:, None, None, None, None]),
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\parallel+J_\parallel\right>$ [$t+J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/l_in_tj.pdf')
plt.show()

In [ ]:
fig = pair_plot(
    y=excited_state_properties.r_conts,
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\perp+J_\perp\right>$ [$t$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/r_in_t.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.r_conts / excited_state_properties.j_to_ts[:, None, None, None, None],
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\perp+J_\perp\right>$ [$J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/r_in_j.pdf')
plt.show()

fig = pair_plot(
    y=excited_state_properties.r_conts / (1 + excited_state_properties.j_to_ts[:, None, None, None, None]),
    ylim=None,
    is_multiplied=False,
    ylabel=r"pair $\Delta\left<t_\perp+J_\perp\right>$ [$t+J$]",
    converge=False,
)
# plt.savefig(notebook_dir+f'pic-exc_pair_analysis/r_in_tj.pdf')
plt.show()

In [ ]:
# Print values of selected combinations.

# ix = 0
# for qx,qy,qpy in ([0,1,1], [1,1,1], [4,1,0]):
#     print_selected(ix,qx,qy,qpy,r_conts,pairs,is_multiplied=False)

# ix = -1
# for qx,qy,qpy in ([2,0,0], [2,0,1], [3,0,0]):
#     print_selected(ix,qx,qy,qpy,r_conts,pairs,is_multiplied=False)

# ix = 0
# sets = [(0,1,0,0),(0,1,1,1),(1,1,0,0),(1,1,1,3),(4,1,0,0),(4,0,1,0)]
# for qx,qy,pm,exc in sets:
#     print(qx,qy,pm,exc, tr_conts[ix,pm,qx,qy,exc])

ix = 3
renorm = 1 / np.sqrt(config.hamiltonian.num_rungs)
print(f"J/t={j_to_ts[ix]}")
data = [
    excited_state_properties.energies,
    excited_state_properties.projected_norms * renorm,
    excited_state_properties.tl_conts,
    excited_state_properties.tr_conts,
    excited_state_properties.jl_conts,
    excited_state_properties.jr_conts,
]
labels = ["energies", "norms", "tl_conts", "tr_conts", "jl_conts", "jr_conts"]
# sets = [(2,0,0,0),(2,0,1,0),(2,1,1,0),(3,0,0,0),(3,0,1,0)] # J = 1.5t
sets = [(1, 1, 0, 0), (1, 1, 1, 1), (2, 0, 0, 0), (2, 1, 1, 0), (3, 0, 0, 0), (3, 0, 1, 0)]  # J = 0.4t
for datum, label in zip(data, labels):
    print(label)
    for qx, qy, pm, exc in sets:
        print(qx, qy, pm, exc, datum[ix, pm, qx, qy, exc])

In [ ]:
# Check the size an excited state projected onto another for various J/t
holes = [5, 3]
# jt1 = 0.4
# jt2 = 0.1
# q_el = [1, 1]
# q_hole = [1, 1]
# exc1 = [0, 1]
# exc2 = [0, 3]

# jt1 = 0.4
# jt2 = 1.5
# q_el = [2, 1]
# q_hole = [2, 0]
# exc1 = [0, 1]
# exc2 = [0, 3]

# jt1 = 0.4
# jt2 = 0.1
# q_el = [1, 1]
# q_hole = [1, 1]
# exc1 = [0, 1]
# exc2 = [0, 3]

jt1 = 1.5
jt2 = 0.1
q_el = [1, 1]
q_hole = [1, 1]
exc1 = [0, 1]
exc2 = [0, 3]


excited_config = copy.deepcopy(config)
for pm, num_holes in enumerate(holes):
    q = q_hole if pm else q_el
    fs.mutate_hamiltonian_config(excited_config.hamiltonian, j_to_t=jt1, num_holes=num_holes, ks=q)
    eigvals1, eigvecs1 = api.get_eigenpairs(excited_config)
    eigvec1 = eigvecs1[:, exc1[pm]]

    fs.mutate_hamiltonian_config(excited_config.hamiltonian, j_to_t=jt2, num_holes=num_holes, ks=q)
    eigvals2, eigvecs2 = api.get_eigenpairs(excited_config)
    eigvec2 = eigvecs2[:, exc2[pm]]

    overlap = np.abs(np.vdot(eigvec1, eigvec2))
    print(f"{num_holes=}, {overlap=}")